In [ ]:
%pip install -U langgraph

In [5]:
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver
import time
import os

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

@task
def call_background_check_api(candidate_id: str) -> dict:
    print(f"--- TASK: Calling background check API for {candidate_id} ($25 charge) ---")
    time.sleep(2)
    return {
        "criminal_record": "clear",
        "employment_verified": True,
    }

@task
def call_reference_check_service(candidate_id: str) -> list:
    print(f"--- TASK: Calling reference check service for {candidate_id} (10 min wait) ---")
    time.sleep(1)
    return [
        {"reference": "Manager A", "rating": 5},
        {"reference": "Colleague B", "rating": 4}
    ]

@entrypoint(checkpointer=MemorySaver())
def complete_screening(candidate_id: str) -> dict:
    print(f"\n--- WORKFLOW: Starting complete screening for {candidate_id} ---")

    background = call_background_check_api(candidate_id).result()
    references = call_reference_check_service(candidate_id).result()

    decision = "Approved" if background["criminal_record"] == "clear" else "Rejected"

    print(f"--- WORKFLOW: Decision for {candidate_id} is {decision} ---")

    return {
        "candidate_id": candidate_id,
        "background": background,
        "references": references,
        "decision": decision
    }

if __name__ == "__main__":
    thread_config = {"thread_id": "screening_007"}

    print("="*60)
    print("RUN 1: Executing all expensive tasks...")
    print("="*60)
    result1 = complete_screening.invoke("CAND_12345", config=thread_config)

    print("\n" + "="*60)
    print("RUN 2: Using cached results from checkpoints...")
    print("="*60)
    result2 = complete_screening.invoke("CAND_12345", config=thread_config)

    print("\n--- FINAL DECISION ---")
    print(result2.get("decision"))

RUN 1: Executing all expensive tasks...

--- WORKFLOW: Starting complete screening for CAND_12345 ---
--- TASK: Calling background check API for CAND_12345 ($25 charge) ---
--- TASK: Calling reference check service for CAND_12345 (10 min wait) ---
--- WORKFLOW: Decision for CAND_12345 is Approved ---

RUN 2: Using cached results from checkpoints...

--- WORKFLOW: Starting complete screening for CAND_12345 ---
--- TASK: Calling background check API for CAND_12345 ($25 charge) ---
--- TASK: Calling reference check service for CAND_12345 (10 min wait) ---
--- WORKFLOW: Decision for CAND_12345 is Approved ---

--- FINAL DECISION ---
Approved
